In [1]:
#Import Dependencies
import pandas as pd
import sqlite3
from sqlite3 import Error


In [3]:
# Read in CSVs

filename = "PlayerBios.csv"
filepath = "../../DataBase/Resources/"
player_df = pd.read_csv(f"{filepath}{filename}")

      

In [4]:
#Check df names
player_df.head()

,Player_ID,Name,rel_link,Position,height,weight (lbs),dob_age,experience,college,height.1,weight_lbs,dob,exp(y),draft_year,draft_round,draft_pick,draft_team
0,21042,A.J. Brown,/nfl/aj-brown-fantasy/21042,WR,"6'0""",226,"June 30, 1997 (25)",4th Season,Mississippi,"6'0""",226,6/30/1997,4,2019,2.0,51.0,TEN
1,12845,A.J. Green,/nfl/aj-green-fantasy/12845,WR,"6'4""",207,"July 31, 1988 (33)",12th Season,Georgia,"6'4""",207,7/31/1988,12,2011,1.0,4.0,CIN
2,16962,AJ Derby,/nfl/aj-derby-fantasy/16962,TE,"6'5""",245,"September 20, 1991 (30)",5th Season,Arkansas,"6'5""",245,9/20/1991,5,2015,6.0,202.0,NE
3,21802,AJ Dillon,/nfl/aj-dillon-fantasy/21802,RB,"6'0""",247,"May 2, 1998 (24)",3rd Season,Boston College,"6'0""",247,5/2/1998,3,2020,2.0,62.0,GB
4,16116,AJ McCarron,/nfl/aj-mccarron-fantasy/16116,QB,"6'3""",215,"September 13, 1990 (31)",8th Season,Alabama,"6'3""",215,9/13/1990,8,2014,5.0,164.0,CIN


In [5]:
player_df.drop(columns="height.1", inplace = True)

Start SQL Imports

In [6]:
#Start SQL connections
con = sqlite3.connect("./sqlite_db.sqlite")

In [7]:
player_df.to_sql("PlayerBios", con, if_exists="replace")

c:\Users\camke\anaconda3\envs\PythonData\lib\site-packages\pandas\core\generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [8]:
cursor = con.cursor()

In [9]:
# Getting all tables from sqlite_master
sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""

In [10]:
# executing our sql query
cursor.execute(sql_query)
print("List of tables\n")
    
# printing all tables list
print(cursor.fetchall())

List of tables

[('Teams',), ('QB_Stats',), ('QB_Extra_Stats',), ('WR_Stats',), ('WR_Extra_Stats',), ('RB_Stats',), ('RB_Extra_Stats',), ('TE_Stats',), ('TE_Extra_Stats',), ('QB_Full_Stats',), ('WR_Full_Stats',), ('RB_Full_Stats',), ('TE_Full_Stats',), ('PlayerBios',)]
